In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/note/LeafDD

/content/drive/MyDrive/note/LeafDD


In [ ]:
!ls

Apple		   datasets	     final.ipynb      ShallowNet.hdf5
Apple.tar.xz	   datasets2	     miniVGGNet.hdf5
CoffeeLeaf.tar.xz  datasets2.tar.xz  modVGGNet.hdf5


In [ ]:
!tar -xvf datasets2.tar.xz 

In [ ]:
!ls datasets2/

'Apple Black rot'	      'Peach Bacterial spot'
'Apple cedar rust'	      'Peach Healthy'
'Apple Healthy'		      'Potato Early blight'
'Apple Scab'		      'Potato Healthy'
'Bell pepper Bacterial spot'  'Potato Late blight'
'Bell pepper Healthy'	      'Tomato Bacterial spot'
'Cherry Healthy'	      'Tomato Early blight'
'Cherry Powdery mildew'       'Tomato Healthy'
'Corn Common rust'	      'Tomato Late blight'
'Corn Gray leaf spot'	      'Tomato Leaf Mold'
'Corn Healthy'		      'Tomato Mosaic virus'
'Corn Northern Leaf Blight'   'Tomato Septoria leaf spot'
'Grape Black Measles'	      'Tomato Spider mites'
'Grape Black rot'	      'Tomato Target Spot'
'Grape Healthy'		      'Tomato Yellow Leaf Curl Virus'
'Grape Isariopsis Leaf Spot'


In [ ]:
import numpy as np
import tensorflow as tf
import cv2, os
from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.utils.image_utils import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
from keras.optimizers import SGD, schedules

In [ ]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        # Lưu ảnh tiền xử lý
        self.preprocessors = preprocessors

        # Nếu bước tiền xử lý là None thì khởi tạo danh sách rỗng
        if self.preprocessors is None:
            self.preprocessors = []

    def load(self, imagePaths, verbose=-1):
        # Khởi tạo danh sách các đặc trưng và nhãn
        data = []
        labels = []

        # Lặp qua tất cả ảnh đầu vào
        for (i, imagePath) in enumerate(imagePaths):
            # Nạp ảnh và trích xuất nhãn từ đường dẫn định dạng
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            # check to see if our preprocessors are not None
            if self.preprocessors is not None:
                # Lặp qua tất cả tiền xử lý và áp dụng cho mỗi ảnh
                for p in self.preprocessors:
                    image = p.preprocess(image)
            # Mỗi ảnh được xử lý là vector đặc trưng bằng cách
            # cập nhật danh sách dữ liệu cùng với nhãn
            data.append(image)
            labels.append(label)

            # Hiển thị ảnh cập nhật
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
               print("[INFO] Đã xử lý {}/{}".format(i + 1,len(imagePaths)))
                # Trả về dữ liệu kiểu tuple gồm dữ liệu và nhãn
        return (np.array(data), np.array(labels))

In [ ]:
class ImageToArrayPreprocessor:  # Tạo lớp để chuyển ảnh --> mảng
    def __init__(self, dataFormat=None):
        # Lưu ảnh đã được định dạng
        self.dataFormat = dataFormat

    def preprocess(self, image): # Định nghĩa phương thức preprocess trả về mảng
        # Hàm img_to_array của Keras
        return img_to_array(image, data_format=self.dataFormat)
class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # Lưu image width, height và interpolation
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # Trả về ảnh có kích thước đã thay đổi
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

In [ ]:
class ShallowNet:
    @staticmethod
    def build(width, height, depth, classes):
        # khởi tạo mô hình
        # height, width, depth: tương ứng 3 chiều của dữ liệu ảnh đầu vào
        # classes: tổng số lớp mà mạng dự đoán, phụ thuộc vào dữ liệu
        # Dữ liệu Animal: 3 lớp; Dữ liệu CIFAR-10: 10 lớp

        model = Sequential()
        inputShape = (height, width, depth)

        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)

        # Định nghĩa mạng CONV => RELU layer
        model.add(Conv2D(32, (3, 3), padding="same",input_shape = inputShape))
        model.add(Activation("relu"))

        # Bộ phân lớp sử dụng hàm softmax
        model.add(Flatten())       # Chuyển thành vector
        model.add(Dense(classes))  # Định nghĩa Full Connected layer
        model.add(Activation("softmax"))   # Kích hoạt hàm softmax để phân lớp
        # Trả về model kiến trúc mạng
        return model

In [ ]:
class MiniVGGNet:
  @staticmethod
  def build(width, height, depth, classes):
      # Khởi tạo mô hình, shape ảnh đầu vào và số kênh của ảnh đầu vào
      model = Sequential()
      input_shape = (height, width, depth)
      channel_dim = -1  # chỉ số của số kênh ảnh đầu vào
                        # giá trị -1 ý muốn nói chỉ số kênh nằm cuối cùng
                        # của danh sách chứa dữ liệu ảnh đầu vào

      # sử dụng 'channels_first' để cập nhật shape và số kênh ảnh đầu vào
      if K.image_data_format() == 'channels_first':
          input_shape = (depth, height, width)
          channel_dim = 1

      # Chuỗi layer đầu tiên  CONV => RELU => CONV => RELU => POOL
      model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(Conv2D(32, (3, 3), padding='same'))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(MaxPooling2D(pool_size=(2, 2)))
      model.add(Dropout(0.25))

      # Chuỗi layer thứ hai CONV => RELU => CONV => RELU => POOL
      model.add(Conv2D(64, (3, 3), padding='same'))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(Conv2D(64, (3, 3), padding='same'))
      model.add(Activation('relu'))
      model.add(BatchNormalization(axis=channel_dim))
      model.add(MaxPooling2D(pool_size=(2, 2)))
      model.add(Dropout(0.25))

      # Thiết lập FC thứ nhất => RELU layers
      model.add(Flatten())
      model.add(Dense(512))
      model.add(Activation('relu'))
      model.add(BatchNormalization())
      model.add(Dropout(0.25))   # Dropout 50%

      # THiết lập FC thứ hai => Hàm phân lớp Softmax
      model.add(Dense(classes))
      model.add(Activation('softmax'))

      # Trả về kiến trúc mạng/mô hình
      return model

In [ ]:
class ModVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        # Khởi tạo mô hình, shape ảnh đầu vào và số kênh của ảnh đầu vào
        model = Sequential()
        input_shape = (height, width, depth)
        channel_dim = -1  # chỉ số của số kênh ảnh đầu vào
                          # giá trị -1 ý muốn nói chỉ số kênh nằm cuối cùng
                          # của danh sách chứa dữ liệu ảnh đầu vào

        # sử dụng 'channels_first' để cập nhật shape và số kênh ảnh đầu vào
        if K.image_data_format() == 'channels_first':
            input_shape = (depth, height, width)
            channel_dim = 1

        # Chuỗi layer đầu tiên  CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(32, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Chuỗi layer thứ hai CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(64, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Chuỗi layer thứ ba CONV => RELU => CONV => RELU => POOL
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(Conv2D(128, (3, 3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization(axis=channel_dim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Thiết lập FC thứ nhất => RELU layers
        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))   # Dropout 50%

        # THiết lập FC thứ hai => Hàm phân lớp Softmax
        model.add(Dense(classes))
        model.add(Activation('softmax'))

        # Trả về kiến trúc mạng/mô hình
        return model

In [ ]:
vb = 64

imagePaths = list(paths.list_images("datasets2/"))

# Bước 1. Chuẩn bị dữ liệu
# Khởi tạo tiền xử lý ảnh
sp = SimplePreprocessor(32, 32) # Thiết lập kích thước ảnh 32 x 32
iap = ImageToArrayPreprocessor() # Gọi hàm để chuyển ảnh sang mảng

# Nạp dataset từ đĩa
print("[INFO] Nạp ảnh...")

sdl = SimpleDatasetLoader(preprocessors=[sp, iap])
(data, labels) = sdl.load(imagePaths, verbose=vb)
data = data.astype("float") / 255.0

[INFO] Nạp ảnh...
[INFO] Đã xử lý 64/25329
[INFO] Đã xử lý 128/25329
[INFO] Đã xử lý 192/25329
[INFO] Đã xử lý 256/25329
[INFO] Đã xử lý 320/25329
[INFO] Đã xử lý 384/25329
[INFO] Đã xử lý 448/25329
[INFO] Đã xử lý 512/25329
[INFO] Đã xử lý 576/25329
[INFO] Đã xử lý 640/25329
[INFO] Đã xử lý 704/25329
[INFO] Đã xử lý 768/25329
[INFO] Đã xử lý 832/25329
[INFO] Đã xử lý 896/25329
[INFO] Đã xử lý 960/25329
[INFO] Đã xử lý 1024/25329
[INFO] Đã xử lý 1088/25329
[INFO] Đã xử lý 1152/25329
[INFO] Đã xử lý 1216/25329
[INFO] Đã xử lý 1280/25329
[INFO] Đã xử lý 1344/25329
[INFO] Đã xử lý 1408/25329
[INFO] Đã xử lý 1472/25329
[INFO] Đã xử lý 1536/25329
[INFO] Đã xử lý 1600/25329
[INFO] Đã xử lý 1664/25329
[INFO] Đã xử lý 1728/25329
[INFO] Đã xử lý 1792/25329
[INFO] Đã xử lý 1856/25329
[INFO] Đã xử lý 1920/25329
[INFO] Đã xử lý 1984/25329
[INFO] Đã xử lý 2048/25329
[INFO] Đã xử lý 2112/25329
[INFO] Đã xử lý 2176/25329
[INFO] Đã xử lý 2240/25329
[INFO] Đã xử lý 2304/25329
[INFO] Đã xử lý 2368/25329

In [ ]:
classLabels = ['Apple Black rot', 'Apple cedar rust', 'Apple Healthy', 'Apple Scab', 'Bell pepper Bacterial spot', 'Bell pepper Healthy',
                'Cherry Healthy', 'Cherry Powdery mildew', 'Corn Common rust', 'Corn Gray leaf spot', 'Corn Healthy', 'Corn Northern Leaf Blight',
                'Grape Black Measles', 'Grape Black rot', 'Grape Healthy', 'Grape Isariopsis Leaf Spot', 'Peach Bacterial spot', 'Peach Healthy',
                'Potato Early blight', 'Potato Healthy', 'Potato Late blight', 'Tomato Bacterial spot', 'Tomato Early blight', 'Tomato Healthy',
                'Tomato Late blight', 'Tomato Leaf Mold', 'Tomato Mosaic virus', 'Tomato Septoria leaf spot', 'Tomato Spider mites', 'Tomato Target Spot',
                'Tomato Yellow Leaf Curl Virus']

# Chia tách dữ liệu vào 02 tập, training: 75% và testing: 25%
(trainX, testX, trainY, testY) = train_test_split(data, labels,test_size=0.25, random_state=42)
# Chuyển dữ liệu nhãn ở số nguyên vào biểu diễn dưới dạng vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=1,)
optimizer = SGD(learning_rate=lr_schedule, momentum=0.9, nesterov=True)

ep = 10

In [ ]:
model = ShallowNet.build(width=32, height=32, depth=3, classes=len(classLabels))
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=ep, verbose=1)

model.save("ShallowNet.hdf5")
# model.summary()

print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classLabels))

[INFO]: Đang trainning....
Epoch 1/10
297/297 [==============================] - 25s 80ms/step - loss: 2.0713 - accuracy: 0.4285 - val_loss: 1.5396 - val_accuracy: 0.5846
Epoch 2/10
297/297 [==============================] - 19s 63ms/step - loss: 1.3335 - accuracy: 0.6337 - val_loss: 1.3131 - val_accuracy: 0.6327
Epoch 3/10
297/297 [==============================] - 19s 65ms/step - loss: 1.0771 - accuracy: 0.6996 - val_loss: 1.1440 - val_accuracy: 0.6766
Epoch 4/10
297/297 [==============================] - 19s 64ms/step - loss: 0.9035 - accuracy: 0.7481 - val_loss: 0.9836 - val_accuracy: 0.7270
Epoch 5/10
297/297 [==============================] - 19s 64ms/step - loss: 0.7740 - accuracy: 0.7839 - val_loss: 1.0296 - val_accuracy: 0.7275
Epoch 6/10
297/297 [==============================] - 20s 66ms/step - loss: 0.6694 - accuracy: 0.8092 - val_loss: 1.0004 - val_accuracy: 0.7199
Epoch 7/10
297/297 [==============================] - 19s 64ms/step - loss: 0.5768 - accuracy: 0.8377 - val_l

In [ ]:
model = MiniVGGNet.build(width=32, height=32, depth=3, classes=len(classLabels))
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=ep, verbose=1)

model.save("miniVGGNet.hdf5")
# model.summary()

print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classLabels))

[INFO]: Đang trainning....
Epoch 1/10
297/297 [==============================] - 168s 562ms/step - loss: 1.5265 - accuracy: 0.5745 - val_loss: 3.9006 - val_accuracy: 0.2678
Epoch 2/10
297/297 [==============================] - 179s 602ms/step - loss: 0.7526 - accuracy: 0.7746 - val_loss: 0.8521 - val_accuracy: 0.7355
Epoch 3/10
297/297 [==============================] - 181s 610ms/step - loss: 0.5418 - accuracy: 0.8314 - val_loss: 0.7402 - val_accuracy: 0.7841
Epoch 4/10
297/297 [==============================] - 170s 573ms/step - loss: 0.4230 - accuracy: 0.8704 - val_loss: 0.4914 - val_accuracy: 0.8487
Epoch 5/10
297/297 [==============================] - 179s 604ms/step - loss: 0.3440 - accuracy: 0.8909 - val_loss: 0.6953 - val_accuracy: 0.7973
Epoch 6/10
297/297 [==============================] - 170s 574ms/step - loss: 0.2666 - accuracy: 0.9174 - val_loss: 0.3840 - val_accuracy: 0.8811
Epoch 7/10
297/297 [==============================] - 173s 581ms/step - loss: 0.2300 - accuracy: 

In [ ]:
model = ModVGGNet.build(width=32, height=32, depth=3, classes=len(classLabels))
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

print("[INFO]: Đang trainning....")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=ep, verbose=1)

model.save("modVGGNet.hdf5")
# model.summary()

print("[INFO]: Đánh giá model....")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=classLabels))

[INFO]: Đang trainning....
Epoch 1/10
297/297 [==============================] - 222s 741ms/step - loss: 1.7528 - accuracy: 0.5301 - val_loss: 5.1148 - val_accuracy: 0.2176
Epoch 2/10
297/297 [==============================] - 222s 748ms/step - loss: 0.8825 - accuracy: 0.7382 - val_loss: 1.5433 - val_accuracy: 0.6011
Epoch 3/10
297/297 [==============================] - 230s 775ms/step - loss: 0.6702 - accuracy: 0.7953 - val_loss: 0.6910 - val_accuracy: 0.7878
Epoch 4/10
297/297 [==============================] - 229s 772ms/step - loss: 0.5280 - accuracy: 0.8383 - val_loss: 0.5403 - val_accuracy: 0.8438
Epoch 5/10
297/297 [==============================] - 224s 755ms/step - loss: 0.4281 - accuracy: 0.8649 - val_loss: 0.4630 - val_accuracy: 0.8642
Epoch 6/10
297/297 [==============================] - 225s 756ms/step - loss: 0.3661 - accuracy: 0.8850 - val_loss: 0.5255 - val_accuracy: 0.8386
Epoch 7/10
297/297 [==============================] - 229s 770ms/step - loss: 0.3193 - accuracy: 